In [1]:
!pip install mesa seaborn --quiet

In [1]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import random

In [2]:
import random

class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.points = 4

    def get_adjacent_agents(self):
        # Obtener los agentes en las celdas adyacentes
        adjacent_agents = []
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)

        for pos in possible_moves:
            adjacent_agents.extend(self.model.grid.get_cell_list_contents([pos]))
        
        return adjacent_agents

    def decide_to_save_points(self):
        # Decidir aleatoriamente si ahorrar puntos
        return random.choice([True, False])  # 50% de probabilidad de ahorrar

    def interact(self):
        adjacent_agents = self.get_adjacent_agents()
        
        for agent in adjacent_agents:
            if isinstance(agent, GreenGhostAgent):
                # Decide si interactuar con el fantasma
                if self.points >= 2:  # Puede exorcizar
                    self.perform_exorcism(agent)
                elif self.points >= 1:  # Puede noquear
                    self.perform_knocked(agent)
            elif isinstance(agent, ArtifactAgent):
                # Decide recoger el artefacto si hay puntos
                if self.points >= 1:
                    self.collect_artifact(agent)

    def move(self):
        if self.points > 0:
            possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            new_position = self.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            self.points -= 1  # Costo de moverse

    def perform_knocked(self, ghost):
        if self.points >= 1:  # Costo de noquear
            ghost.knocked = True
            self.points -= 1
            print(f"Luigi {self.unique_id} ha noqueado un fantasma en {self.pos}")

    def perform_exorcism(self, ghost):
        if self.points >= 2:  # Costo de exorcizar
            ghost.absorb = True
            self.points -= 2
            print(f"Luigi {self.unique_id} ha absorbido un fantasma en {self.pos}")

    def collect_artifact(self, artifact):
        artifact.recovered = True
        self.points -= 1  # Costo de recoger el artefacto
        print(f"Luigi {self.unique_id} ha recogido un artefacto en {self.pos}")

    def step(self):
        if not self.decide_to_save_points():
            self.move()  # Si no decide ahorrar, se mueve
        self.interact()  # Realiza interacciones




In [ ]:
class GreenGhostAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

In [ ]:
class RedGhostAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

In [ ]:
class ArtifactAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

In [ ]:
class MansionModel(Model):
    def __init__(self, width, height, num_agents):
        self.num_agents = num_agents
        self.grid = SingleGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.datacollector = DataCollector(model_reporters={"Damage": lambda m: m.total_damage})

        for i in range(self.num_agents):
            luigi = LuigiAgent(i, self)
            self.schedule.add(luigi)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(luigi, (x, y))

        self.total_damage = 0 #Este es el da;o de la casa 

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)